# ユーザの追加

---
OpenHPC環境にユーザを追加します。

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* OpenHPC環境のマスターノード、計算ノードに対してAnsibleで操作できるように設定されている

VCノードを作成時に指定した値を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

各ノードに対してAnsibleによる操作が行えることを確認します。操作対象となる UnitGroup 名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

マスターノードに対応する Ansible のグループ名を指定します。

In [ ]:
target_master = f'{ugroup_name}_master'

計算ノードに対応する Ansible のグループ名を指定します。

In [ ]:
target_compute = f'{ugroup_name}_compute'

疎通確認を行います。

In [ ]:
!ansible {target_master} -m ping
!ansible {target_compute} -m ping

管理者権限でコマンドを実行できることを確認します。

In [ ]:
!ansible {target_master} -b -a 'whoami'
!ansible {target_compute} -b -a 'whoami'

## ユーザ登録

OpenHPC環境にユーザを登録します。

追加するユーザ名を指定してください。

In [ ]:
# (例)
# user = 'user00'

user = 

ユーザの新規登録を行います。

In [ ]:
!ansible {target_master} -b -m user -a 'name={user}'
!ansible {target_compute} -b -m user -a 'name={user}'

作成したユーザのSSHの公開鍵を `$HOME/.ssh/authorized_keys` に追加します。

次のセルを実行すると入力枠が表示されるので、SSHの公開鍵を入力してください（最後にEnterキーをタイプすることで入力内容が確定します）。

In [ ]:
from getpass import getpass
ssh_public_key = getpass()

入力されたSSHの公開鍵を登録します。

In [ ]:
!ansible {target_master} -b -m authorized_key -a 'user={user} key="{ssh_public_key}"'

試しに登録したユーザでログインして、コマンドを実行してみます。SSHの秘密鍵とログインするホストのIPアドレスを設定してから、次のセルを実行してください。

In [ ]:
# 秘密鍵のパス
# (例)
# path_identity = '~/.ssh/id_rsa'
path_identity = 

# ログインするホストのIPアドレス
# (例)
# target_ipaddr = '172.30.2.120'
target_ipaddr = 

!ssh -i {path_identity} {user}@{target_ipaddr} hostname

## グループの追加

ユーザをプライマリグループ(メイングループ)以外のグループに登録する場合はこの節を実行してください。

> 登録ユーザに docker コマンドの実行を許可する場合は `docker` グループへの登録が必要となります。

登録するグループのリストを次のセルで指定してください。

In [ ]:
groups = [
# (例)
#    'docker',

]

ユーザを指定したグループに追加します。

In [ ]:
if len(groups) > 0:
    !ansible {target_master} -b -m user -a 'name={user} append=yes groups={",".join(groups)}'
    !ansible {target_compute} -b -m user -a 'name={user} append=yes groups={",".join(groups)}'

ユーザがグループに登録されたことを確認します。

In [ ]:
!ssh -i {path_identity} {user}@{target_ipaddr} id